# Essais de requêtes

## Accession
Aller chercher un numéro d'accession pour l'espèce qui servira d'entrée pour trouver la BdD ensembl appropriée

In [48]:
# !/usr/bin/ python3
#-*- coding : utf-8 -*- 
import requests, sys

## Fonction pour requête API et sortie en Json
def RequestJson (server, ext):
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
    
    #Retour en cas d'erreur
    if not r.ok:
        r.raise_for_status()
        sys.exit()

    return(r.json())

## Fonction pour récupérer la division Ensembl en cas d'échec de la méthode principale
def RetrieveDivAlt(species) :

    # Retrouve les divisions ensembl 
    server = "https://rest.ensembl.org"
    ext = "/info/divisions?" 
 
    divisions = RequestJson(server,ext)[::-1] # Inverse pour chercher dans les espèce bactériennes d'abord

    # Retrouve la division associé à l'espèce
    j = 0
    div = None
    while not div and j<len(divisions) :
        
        ext = f"/info/species?division={divisions[j]}"     
        decoded = RequestJson(server,ext)

        for i in range(len(decoded["species"])) :
            if species in decoded["species"][i]["name"]:
                div = decoded["species"][i]["division"]
        j+=1
    return(div)

## Fonction pour récupérer la division Ensembl 
def RetrieveDiv (species) :
    # Requête des infos concernant l'espèce
    server = "https://rest.ensembl.org" # Serveur Ensembl pour requête API
    ext = f"/info/assembly/{species}?" # Requête d'un assemblage de l'espèce
    decoded = RequestJson(server, ext) # Résultat requête

    accession = decoded['assembly_accession'] # Récupération d'une accession d'un génome    

    # Requête des infos concernant l'accession
    ext = f"/info/genomes/assembly/{accession}?"
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"}) 

    # Retour en cas d'erreur
    if not r.ok:
        division = RetrieveDivAlt(species)
    else : 
        division = r.json()["division"] # Résultat API

    if division[7:].lower() == "vertebrates" :
        div = "www"
    else : 
        div = division[7:].lower()
    
    return(div)

## Fonction pour récupérer les informations relatives au symbole dans l'espèce spécifiée
def InfoGene (species, symbol) :
    # Serveur Ensembl pour requête API
    server = "https://rest.ensembl.org"

    # Requête vers l'espèce et symbole de gène courants
    ext = f"/lookup/symbol/{species}/{symbol}?expand=1"     
    decoded = RequestJson(server,ext)
    
    # Construction du dictionnaire contenant les infos
    species_info = {}
    species_info["gene_symbol"] = symbol
    species_info["gene_id"] = decoded["id"]
    species_info["division"] = RetrieveDiv(species)

    for i in decoded["Transcript"] :
        if "transcript_id" not in species_info :
            species_info["transcript_id"] = i["id"]
            species_info["prot_id"] = i["Translation"]["id"]
        else :
            # Formatage pour le HTML 
            species_info["transcript_id"] += "<br>\n\t\t\t" + i["id"]
            species_info["prot_id"] += "<br>\n\t\t\t" + i["Translation"]["id"]

    species_info["gene_browser"] = f"https://{species_info["division"]}.ensembl.org/{species.capitalize()}/Location/View?db=core;g={species_info["gene_id"]}"
    return(species_info)

with open("GeneSymbols_45.txt", "r") as infos:
    for line in infos:
        symbol = line.split(",")[0]
        current_species = line[:-1].split(",")[1]
        print(InfoGene(current_species, symbol)["gene_browser"])


https://www.ensembl.org/Homo_sapiens/Location/View?db=core;g=ENSG00000051180
https://plants.ensembl.org/Arabidopsis_thaliana/Location/View?db=core;g=AT2G18790
https://fungi.ensembl.org/Saccharomyces_cerevisiae/Location/View?db=core;g=YDL168W
https://protists.ensembl.org/Leishmania_major/Location/View?db=core;g=LMJF_27_0290
https://bacteria.ensembl.org/Escherichia_coli_gca_006381815/Location/View?db=core;g=ENSB:fqjn3NbmMv9mSox


## Création HTML
écrire structure et modifier html

In [22]:
dic = {'homo_sapiens': [{'symbol': 'RAD51', 'gene_id': 'ENSG00000051180', 'transcript_id': 'ENST00000527860<br>\n\t\t\tENST00000423169<br>\n\t\t\tENST00000557850<br>\n\t\t\tENST00000382643<br>\n\t\t\tENST00000267868<br>\n\t\t\tENST00000645673<br>\n\t\t\tENST00000525066<br>\n\t\t\tENST00000526763<br>\n\t\t\tENST00000532743<br>\n\t\t\tENST00000531277<br>\n\t\t\tENST00000533741', 'prot_id': 'ENSP00000432759<br>\n\t\t\tENSP00000406602<br>\n\t\t\tENSP00000454176<br>\n\t\t\tENSP00000372088<br>\n\t\t\tENSP00000267868<br>\n\t\t\tENSP00000493712<br>\n\t\t\tENSP00000431864<br>\n\t\t\tENSP00000431897<br>\n\t\t\tENSP00000433924<br>\n\t\t\tENSP00000436512<br>\n\t\t\tENSP00000433053'}], 'arabidopsis_thaliana': [{'symbol': 'PHYB', 'gene_id': 'AT2G18790', 'transcript_id': 'AT2G18790.2<br>\n\t\t\tAT2G18790.1', 'prot_id': 'AT2G18790.2<br>\n\t\t\tAT2G18790.1'}], 'saccharomyces_cerevisiae': [{'symbol': 'SFA1', 'gene_id': 'YDL168W', 'transcript_id': 'YDL168W_mRNA', 'prot_id': 'YDL168W'}], 'Leishmania_major': [{'symbol': 'ACP', 'gene_id': 'LMJF_27_0290', 'transcript_id': 'CBZ12089', 'prot_id': 'CBZ12089'}], 'escherichia_coli_gca_006381815': [{'symbol': 'HYBE', 'gene_id': 'ENSB:fqjn3NbmMv9mSox', 'transcript_id': 'ENSB:fqjn3NbmMv9mSox', 'prot_id': 'ENSB:fqjn3NbmMv9mSox'}]}
html = open("EMBL.html","w")

## HTML Head
head_html = """
<!DOCTYPE HTML>
<html lang="fr-FR">
	<head>
		<title> Formulaire javascript 1 </title>
		<meta charset=UTF-8/>
        <meta name="viewport" content="width=device-width, initial-scale=1.0">

        <!-- CSS -->
        <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.20/css/jquery.dataTables.min.css">
        <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/fixedcolumns/3.3.0/css/fixedColumns.dataTables.min.css">
        <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/buttons/1.7.1/css/buttons.dataTables.min.css">
        <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/fixedheader/3.1.9/css/fixedHeader.dataTables.min.css">
    
        <!-- JavaScript -->
        <script type="text/javascript" language="javascript" src="https://code.jquery.com/jquery-3.7.1.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/1.10.20/js/jquery.dataTables.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/fixedcolumns/3.3.0/js/dataTables.fixedColumns.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/buttons/1.7.1/js/dataTables.buttons.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/buttons/1.7.1/js/buttons.colVis.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/fixedheader/3.1.9/js/dataTables.fixedHeader.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/rowgroup/1.1.2/js/dataTables.rowGroup.min.js"></script>
        
        <!-- Script Supplémentaire-->
        <script type="text/javascript" class="init">
            $(document).ready(function() {
                const table = $('#gene').DataTable({
                    "scrollY": 700,
                    "scrollX": true,
                    lengthMenu: [
                        [5, 10, 25, 50, -1],
                        [5, 10, 25, 50, 'All']
                    ],
                    fixedColumns: {
                        leftColumns: 2
                    },
                    fixedHeader: true,
                    dom: 'Blfrtip',
                    buttons: [
                        'colvis'
                    ]
                });
            });
            </script>
    </head>
    <body>
        <h1>Resultat du scripting d'aggrégation automatique des annotations</h1>

        <table id="gene" class="display nowrap cell-border" style="width:100%">
            <thead>
                <tr>
                    <th><div class=header_1>Species</div></th>
                    <th>Gene Symbol</th>
                    <th>Gene - EMBL ID</th>
                    <th>Protein - EMBL ID</th>
                    <th>Transcript - EMBL ID</th>
                </tr>
            </thead>
            <tbody>
"""

## HTML Body
body_html =""
for esp in dic :
    champ = dic[esp][0]
    body_html += f"""
    <tr>                    
        <td><div class=header_1>
            {esp}
        </div></td>
        <td>
            {champ["symbol"]}
        </td>
        <td>
            {champ["gene_id"]}
        </td>
        <td>
            {champ["prot_id"]}
        </td>
        <td>
            {champ["transcript_id"]}
        </td>
    </tr>"""
tail_html = f""" 
            </tbody>
        </table>
    </body>
</html>
"""

html = open("EMBL.html","w")
html.write(head_html[1:]+body_html+tail_html)
html.close()


## Sauvegarde
Y'a des bouts de code utiles  
```python  
decoded = (json.dumps(decoded,indent=2))  
```